In [2]:
from AHI_AC import *
res = 0.02

row_AHI = round(120/res)
col_AHI = round(120/res)

AHI_lat = np.linspace(60 - res/2 ,-60 + res/2,row_AHI)
AHI_lon = np.linspace(85 + res/2 ,205 - res/2,col_AHI)

target ='/data01/GEO/SRDATA/'

AHI_VZA,AHI_VAA = read_AHI_VA(res)
Landmask = read_Landmask(res)
AHI_AL = read_AHI_AL(res)
Water_idx = get_water_idx_2(Landmask)
interpolation = LUT_interpolator(5)
fn1_5, fn2_5, fn3_5 = interpolation._interpolate()
interpolation = LUT_interpolator(6)
fn1_6, fn2_6, fn3_6 = interpolation._interpolate()


In [3]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI
    print(f"Start download AHI for Band{band}")
    AHI_data = H8_Process('liwei', 'liwei00', band_type[band], band_num[band], date, folder_original)
    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")

    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(6000))
    SR = np.full((6000, 6000), np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')

    # Save file and remove download input data
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(6000, 6000)
    SR = np.around(SR, 4) * 10000

    SR = np.where(SR >= 32767, 32767, SR)
    SR = np.where(SR <= -32767, -32767, SR)
    SR = np.where(np.isnan(SR), -32768, SR)

    SR_file = open(folder_AC + f'H08_{{YYYY}}{{MM}}{{DD}}_{{HH}}{{MIN}}_B{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()

    print(f'Save Band{band} SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:.1f} secs, {:.1f} mins, {:.1f} hours\n\n'.format(band, TIME, TIME / 60, TIME / 3600))


# Set necessary parameters
band_type = {'5': 'sir', '6': 'sir', }
band_num = {'5': '01', '6': '02'}
calculate_6s = {'5': calculate_6s_band5, '6': calculate_6s_band6}
fn1 = {'5': fn1_5, '6': fn1_6}
fn2 = {'5': fn2_5, '6': fn2_6}
fn3 = {'5': fn3_5, '6': fn3_6}


In [2]:

def SR_Band5():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','sir','01',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band5)(fn1_5, fn2_5, fn3_5,Landmask,Aerosol_type,WV,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(6000))
    SR = np.full((6000,6000),np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(6000,6000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B05_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band5 SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band5 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))
    
    
def SR_Band6():
    start_time = T.time()
    # Download AHI
    print("Start download AHI")
    AHI_data = H8_Process('liwei','liwei00','sir','02',date,folder_original)
    if AHI_data == 'No data':
        # date_time_now = date_time_now + date_t
        # remove_original_file(folder_original)
        print("No AHI data, pass")
        return None
    E_Read_AHI_time = T.time()
    print("Finish download AHI")
    
    # SR calculate
    print('Start Calculate SR')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=30)(delayed(calculate_6s_band6)(fn1_6, fn2_6, fn3_6,Landmask,Aerosol_type,WV,AOT550,RAA,AHI_SZA,AHI_VZA,AHI_AL,AHI_data,i) for i in range(6000))
    SR = np.full((6000,6000),np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print('Calculate SR finished')

    # Save file and remove download input data
    print('Start Save SR file ')
    S_SAVE_time = T.time()
    SR=np.array(SR).reshape(6000,6000)
    SR = np.around(SR,4) * 10000

    SR = np.where(SR>=32767,32767,SR)
    SR = np.where(SR<=-32767,-32767,SR)
    SR = np.where(np.isnan(SR),-32768,SR)
    
    SR_file=open(folder_AC+'H08_{}{}{}_{}{}_B06_SR.dat'.format(YYYY,MM,DD,HH,MIN),'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()
    
    print('Save Band6 SR finished')
    remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time=T.time()
    TIME=end_time-start_time
    print('Band6 cost time: {:.1f} secs, {:.1f} mins,{:.1f} hours\n\n'.format(TIME,TIME/60,TIME/3600))

In [5]:
njob = 15
date_start = '2019-07-31 00:10'
date_end = '2019-07-31 23:50'
date_t = dt.timedelta(minutes=30)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
# CAMS_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS()
# CAMS_AERO_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS_AERO()
while date_time_now <= date_e:

    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
    hour = int(date[8:10])

    if hour <= 14 or hour >= 20:  
        print("start processing {}".format(date))
        # make dir
        folder_original = target + date +'_original/'
        folder_AC = target+'{}/{}/{}/'.format(YYYY,MM,DD)
        mkdir(folder_original)
        mkdir(folder_AC)

        S_Read_AHI_time = T.time()

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()

        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
        Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
        print('Finish read CAMS')

        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}'.format(E_ATMOS_time-S_ATMOS_time))

        params = {
                    '5': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '6': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                }
        
        for band in ['5', '6']:
            SR_Process(band)

    date_time_now = date_time_now + date_t

Start 2019-07-31 
start processing 201907310010
Start read solar angle
Finish read solar angle
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 2.1028337478637695
Start download AHI for Band5
Finish download AHI for Band5
Start Calculate SR for Band5


<ipython-input-3-b9bb3bcc3620>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR for Band5 finished
Start Save SR file for Band5
Save Band5 SR finished
Band5 cost time: 33.4 secs, 0.6 mins, 0.0 hours


Start download AHI for Band6
Finish download AHI for Band6
Start Calculate SR for Band6



KeyboardInterrupt

